## Created by yunsuxiaozi 2024/3/21

### 这里主要实现的是SM4算法,这个算法和之前实现的sha256和SM3算法差别有点大.具体实现的代码可以参考PDF来实现,PDF中没有提到对字符串进行padding的方法,网上查到主流的算法有PKCS5padding和PKCS7padding算法,但是我这里没有实现.笔者对于算法也没有深入了解,目前只是实现了算法.本文后面也有验证我的算法正确性的方法.

### 首先是我实现的算法,这里不多做解释.

In [1]:
"""
^(按位与)的python符号是&.
√(按位或)的python符号是|
⊕(异或)的python符号是^.
"""
inf=0xFFFFFFFF#32位无符号整数的最大值(2^32-1)
#循环左移  x=[a1,a2,……,a32] 
def ROTL(x, n):
    x=x&inf#如果输入的数据太大,会限制到2^32-1的范围内.
    n=n%32#如果n>32,则限制到32以内的范围.
    #x=[a1,a2,……,a32]  则x>>(32-n) =[0,0,……,0,a1,a2,……,an] 
    #x<<(n) =[an+1,……a32,0,0,……,0]
    x = ((x << n) | (x >> (32 - n)))&inf
    return x
def T(x):
    x=x&inf
    return L(tau(x))
def T_(x):
    x=x&inf
    return L_(tau(x))  
#系统参数FK
FK=[0xA3B1BAC6,0x56AA3350,0x677D9197,0xB27022DC]
#S盒
s_box=['d690e9fecce13db716b614c228fb2c05',
       '2b679a762abe04c3aa44132649860699',
       '9c4250f491ef987a33540b43edcfac62',
       'e4b31ca9c908e89580df94fa758f3fa6',
       '4707a7fcf37317ba83593c19e6854fa8',
       '686b81b27164da8bf8eb0f4b70569d35',
       '1e240e5e6358d1a225227c3b01217887',
       'd40046579fd327524c3602e7a0c4c89e',
       'eabf8ad240c738b5a3f7f2cef96115a1',
       'e0ae5da49b341a55ad933230f58cb1e3',
       '1df6e22e8266ca60c02923ab0d534e6f',
       'd5db3745defd8e2f03ff6a726d6c5b51',
       '8d1baf92bbddbc7f11d95c411f105ad8',
       '0ac13188a5cd7bbd2d74d012b8e5b4b0',
       '8969974a0c96777e65b9f109c56ec684',
       '18f07dec3adc4d2079ee5f3ed7cb3948']

# ck=[]
# for i in range(32):
#     cki=[]
#     for j in range(4):
#         cki.append('{:02x}'.format(((4*i+j)*7)%256))
#     ck.append(int(''.join(cki),16))
#固定参数CK的取值方法
CK=[0x00070e15, 0x1c232a31, 0x383f464d, 0x545b6269, 
0x70777e85, 0x8c939aa1, 0xa8afb6bd, 0xc4cbd2d9, 
0xe0e7eef5, 0xfc030a11, 0x181f262d, 0x343b4249, 
0x50575e65, 0x6c737a81, 0x888f969d, 0xa4abb2b9, 
0xc0c7ced5, 0xdce3eaf1, 0xf8ff060d, 0x141b2229, 
0x30373e45, 0x4c535a61, 0x686f767d, 0x848b9299, 
0xa0a7aeb5, 0xbcc3cad1, 0xd8dfe6ed, 0xf4fb0209, 
0x10171e25, 0x2c333a41, 0x484f565d, 0x646b7279]
def L(B):
    B=B&inf
    return B^ROTL(B,2)^ROTL(B,10)^ROTL(B,18)^ROTL(B,24)
#L'(B)函数
def L_(B):#13+23=36
    B=B&inf
    return B^ROTL(B,13)^ROTL(B,23)

block_size=32#origin_str应该以128比特为块进行处理,先要进行padding.

def tau(A):
    A=A&inf
    A='{:032b}'.format(A)
    b=""
    for idx in range(4):
        a_idx=A[idx*8:(idx+1)*8]
        a_idx1,a_idx2=int(a_idx[:4], 2),int(a_idx[4:],2)
        b+=(s_box[a_idx1][2*a_idx2:2*(a_idx2+1)])
    return int(b,16)
    
#轮函数F
def F(x0,x1,x2,x3,rk):
    x0=x0&inf
    x1=x1&inf
    x2=x2&inf
    x3=x3&inf
    rk=rk&inf
    return x0^T(x1^x2^x3^rk)
def round_key(MK):
    K=[0]*36
    K[0],K[1]=int(MK[:block_size//4],16)^FK[0],int(MK[block_size//4:block_size//2], 16)^FK[1]
    K[2],K[3]=int(MK[block_size//2:3*block_size//4],16)^FK[2],int(MK[3*block_size//4:], 16)^FK[3]
    #前4个没有问题
    for idx in range(32):
        K[idx+4]=(K[idx]^T_(K[idx+1]^K[idx+2]^K[idx+3]^CK[idx]))
    return K[4:]
    
#这里实现SM4算法的主体框架,SM4算法分组长度为128比特,即16个字节,每个字节对应2个十六进制字符.即32个字符为一组.
def SM4_encode(key='0123456789abcdeffedcba9876543210',origin_str="hello"):#加密秘钥和明文的16进制表示
    #padding的问题到时候处理.
    padding_size=(block_size-len(origin_str)%block_size)%block_size
    encode_str=""
    for idx in range(0,len(origin_str),block_size):
        group=origin_str[idx:idx+block_size]#一组128比特的数据
        X=[0]*36
        X[0],X[1]=int(group[:block_size//4],16),int(group[block_size//4:block_size//2],16)
        X[2],X[3]=int(group[block_size//2:3*block_size//4], 16),int(group[3*block_size//4:], 16)
        rk=round_key(key)
        for idx in range(len(rk)):
            X[idx+4]=F(X[idx],X[idx+1],X[idx+2],X[idx+3],rk[idx])
        y=X[32:][::-1]
        for i in range(len(y)):
            y[i]=str(hex(y[i]))[2:]
            if len(y[i])<8:
                y[i]=(8-len(y[i]))*"0"+y[i]
        encode_str+="".join(y)
    return encode_str

### 这次没有随机生成一大批样例和标准的算法框架比较看输出结果是否相同,因为没找到现成的框架.问了一些AI也没给出满意的答案.

### 这里验证算法准确性的方法是通过和我学习的代码进行比较,用PDF的样例加密100000次(考虑内存问题),看输出结果是否相同.

### 学习链接如下:https://blog.csdn.net/qq_33439662/article/details/125461587


In [2]:
def sbox(a):
    s=['d690e9fecce13db716b614c228fb2c05',
       '2b679a762abe04c3aa44132649860699',
       '9c4250f491ef987a33540b43edcfac62',
       'e4b31ca9c908e89580df94fa758f3fa6',
       '4707a7fcf37317ba83593c19e6854fa8',
       '686b81b27164da8bf8eb0f4b70569d35',
       '1e240e5e6358d1a225227c3b01217887',
       'd40046579fd327524c3602e7a0c4c89e',
       'eabf8ad240c738b5a3f7f2cef96115a1',
       'e0ae5da49b341a55ad933230f58cb1e3',
       '1df6e22e8266ca60c02923ab0d534e6f',
       'd5db3745defd8e2f03ff6a726d6c5b51',
       '8d1baf92bbddbc7f11d95c411f105ad8',
       '0ac13188a5cd7bbd2d74d012b8e5b4b0',
       '8969974a0c96777e65b9f109c56ec684',
       '18f07dec3adc4d2079ee5f3ed7cb3948']
    b=''
    for i in range(4):
        r=int(a[2*i],16)
        c=int(a[2*i+1],16)
        b=b+s[r][c*2:(c+1)*2]
    return int(b,16)

def cl(x,i):#Cycle left
    x='{:032b}'.format(x)
    i=i%32
    x=x[i:]+x[:i]
    return int(x,2)

fk=[int('A3B1BAC6',16),int('56AA3350',16),int('677D9197',16),int('B27022DC',16)]
ck=[]
for i in range(32):
    cki=[]
    for j in range(4):
        cki.append('{:02x}'.format(((4*i+j)*7)%256))
    ck.append(int(''.join(cki),16))
'''
ck=['00070e15','1c232a31','383f464d','545b6269',
    '70777e85','8c939aa1','a8afb6bd','c4cbd2d9',
    'e0e7eef5','fc030a11','181f262d','343b4249',
    '50575e65','6c737a81','888f969d','a4abb2b9',
    'c0c7ced5','dce3eaf1','f8ff060d','141b2229',
    '30373e45','4c535a61','686f767d','848b9299',
    'a0a7aeb5','bcc3cad1','d8dfe6ed','f4fb0209',
    '10171e25','2c333a41','484f565d','646b7279']
'''
def f(x,rk,i):
    return x[i]^t(x[i+1]^x[i+2]^x[i+3]^rk)

def t(x):
    return l(tau(x))
def t_(x):#t'
    return l_(tau(x))

def tau(a):
    a='{:08x}'.format(a)
    b=sbox(a)
    return b

def l(b):
    c=b^cl(b, 2)^cl(b, 10)^cl(b, 18)^cl(b, 24)
    return c
def l_(b):#l'
    c=b^cl(b, 13)^cl(b, 23)
    return c

def enc(x,mk):
    if len(x)!=32:
        x=(32-len(x))*'0'+x
    p=[x[:8],x[8:16],x[16:24],x[24:]]
    x=p
    for i in range(len(x)):
        x[i]=int(x[i],16)
    rk=kea(mk, fk, ck)
    for i in range(32):
        x.append(f(x,rk[i],i))
    y=x[32:][::-1]
    for i in range(len(y)):
        y[i]='{:08x}'.format(y[i])
    y=''.join(y)
    return y

def kea(mk,fk,ck):#Key extension algorithm
    if len(mk)!=32:
        mk=(32-len(mk))*'0'+mk
    p=[mk[:8],mk[8:16],mk[16:24],mk[24:]]
    mk=p
    for i in range(len(mk)):
        mk[i]=int(mk[i],16)
    k=[]
    for i in range(4):
        k.append(mk[i]^fk[i])
    for i in range(32):
        k.append(k[i]^t_(k[i+1]^k[i+2]^k[i+3]^ck[i]))
    rk=k[4:]
    return rk

In [3]:
num_epochs=100000#加密100000次
print(f"my algorithm")
my_list=[]#我自己加密的结果
origin_str='0123456789abcdeffedcba9876543210'
for i in range(num_epochs):
    origin_str=SM4_encode(key='0123456789abcdeffedcba9876543210',origin_str=origin_str)
    my_list.append(origin_str)

print(f"other algorithm")
other_list=[]#别人代码的加密结果
p='0123456789abcdeffedcba9876543210'
for i in range(num_epochs):
    mk='0123456789abcdeffedcba9876543210'
    y=enc(p, mk)
    encoder_str=""
    for j in range(4):
        encoder_str+=(y[j*8:(j+1)*8])
    p=encoder_str
    other_list.append(p)
    
acc_cnt=0
for i in range(num_epochs):
    acc_cnt+=(my_list[i]==other_list[i])
print(f"accuracy:{acc_cnt/num_epochs}")

my algorithm
other algorithm
accuracy:1.0
